In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm

def get_page_content(url: str):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        return soup


def get_pages_urls(url: str,
                   num_pages: int=2)->list:
    """
    Get pages urls for each subtype
    """
    url_list = []
    for i in range(1, num_pages+1):
        url_list.append(url[:-4] + f'/trang-{i}.htm')
    return url_list

In [ ]:
def get_news_urls(url_list: list)->list:

    """
    Get news/article urls for each subtype
    """
    lst = []
    for url in url_list:
        content = get_page_content(url)
        try:
            news_titles = content.find_all('h3', class_='article-title')
            for title in news_titles:

                news_url = requests.compat.urljoin(url, title.a.get('href'))
                lst.append(news_url)
        except:
            pass
    return lst

In [ ]:
def crawl(url: str,
          num_pages: int):
    content = get_page_content('https://dantri.com.vn')
    links = content.body.nav.ol.find_all('a', href=True)
    news_urls = []
    for l in tqdm(links, desc='Collecting article urls'):
        if l.get('href').endswith('.htm') and not l.get('href').startswith('https') and l.get('href').count('/') > 1:
            full_url = requests.compat.urljoin(url, l.get('href')) # Get url of subtype page
            full_url_pages = get_pages_urls(full_url, num_pages) # Get article urls
            news_urls.extend(get_news_urls(full_url_pages))
    return news_urls

In [ ]:
def create_df(urls):
    data = {
        'type': [],
        'title': [],
        'date': [],
        'time': [],
        'content': [],
        'author': []
    }

    for u in tqdm(urls, desc='Collecting data'):
        content = get_page_content(u)
        container = content.main.find('div', class_='body-container')
        if container is not None:
            gr_container = container.find('div', class_='grid-container')
            article = gr_container.div.article

            date = article.find('time', class_='author-time').text
            date, time = date.split('-')

            try:
                data['type'].append(u.split('/')[3])
            except:
                data['type'].append('Not available')

            try:
                data['title'].append(article.h1.text)
            except:
                data['title'].append('Not available')

            try:
                data['date'].append(date.strip())
            except:
                data['date'].append('Not available')

            try:
                data['time'].append(time.strip())
            except:
                data['time'].append('Not available')

            try:
                data['author'].append(article.find('div', class_='author-name').b.text)
            except:
                data['author'].append('Not available')

            try:
                data['content'].append(article.find('h2').text)
            except:
                data['content'].append('Not available')

    df = pd.DataFrame(data)
    return df

In [ ]:
def run(url, num_pages=5):
    urls = crawl(url, num_pages)
    df = create_df(urls)
    return df

In [ ]:
df = run('https://dantri.com.vn', 5)

In [ ]:
result = pd.DataFrame(df)

In [ ]:
result.to_csv('/content/dantri.csv')

In [ ]:
from google.colab import files
files.download("/content/dantri.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
result

,type,title,date,time,content,author
0,kinh-doanh,"Giá vàng nhẫn vẫn ""đu đỉnh"" 83 triệu đồng/lượng","Thứ hai, 30/09/2024",07:11,(Dân trí) - Mặc dù giá vàng thế giới đã hạ nhi...,Nhật Quang
1,kinh-doanh,Người mua vàng đang lời nhanh chưa từng có,"Chủ nhật, 29/09/2024",18:44,(Dân trí) - Giá vàng tăng kỷ lục giúp nhiều ng...,Huỳnh Anh
2,kinh-doanh,"Tuần lập kỷ lục liên tục của giá vàng, bất ngờ...","Chủ nhật, 29/09/2024",08:45,(Dân trí) - Giá vàng nhẫn đạt đỉnh lịch sử tro...,Mỹ Tâm
3,kinh-doanh,"Bitcoin tăng mạnh, vượt 66.000 USD","Thứ bảy, 28/09/2024",10:10,"Bitcoin đã vượt mốc 66.000 USD, mức cao nhất t...",Not available
4,kinh-doanh,"Vàng nhẫn ""cháy hàng"" khi giá lập kỷ lục, mua ...","Thứ bảy, 28/09/2024",09:56,(Dân trí) - Hàng loạt hệ thống kinh doanh vàng...,Mỹ Tâm
...,...,...,...,...,...,...
6465,phap-luat,Tiếp tục tạm dừng phiên xử vụ gây thiệt hại ch...,"Thứ hai, 12/08/2024",13:42,(Dân trí) - Phiên xét xử 6 người bị cáo buộc g...,Bảo Kỳ
6466,phap-luat,"Kẻ sát hại cô gái 17 tuổi, phân xác phi tang ở...","Chủ nhật, 11/08/2024",07:20,"(Dân trí) - Tạ Duy Khanh (39 tuổi, quê Thái Bì...",Nguyễn Hải
6467,phap-luat,Cựu Tổng giám đốc Công ty Coma 18 lĩnh án 8 nă...,"Thứ sáu, 09/08/2024",18:09,"(Dân trí) - HĐXX đánh giá, cựu Tổng giám đốc C...",Nguyễn Hải
6468,phap-luat,"Nam thanh niên lên kịch bản, phân vai diễn hòn...","Thứ sáu, 09/08/2024",17:19,"(Dân trí) - Mâu thuẫn tình cảm với bạn gái, Ng...",Nguyễn Hải
